In [ ]:
# Yerbol Aussat (SID: 20698564)
# CS698, Assignment 4
# Gaussian Mixture Model
import numpy as np
from sklearn.decomposition import PCA
import math

In [ ]:
# importing and normalizing training data
trn_data = np.genfromtxt('/Users/yerbol/Desktop/CS698/assignments/hw2/MNIST_X_train.csv', delimiter=',')
trn_data = trn_data[:45000]

# row_sums = trn_data.sum(axis=1)
# trn_data = trn_data / row_sums[:, np.newaxis]

print trn_data.shape

In [ ]:
pca = PCA(n_components=100)
trn_data = pca.fit_transform(trn_data)
print trn_data.shape

In [ ]:
# importing and normalizing training labels
trn_labels = np.genfromtxt('/Users/yerbol/Desktop/CS698/assignments/hw2/MNIST_y_train.csv', delimiter=',')
trn_labels = trn_labels[:45000]
print trn_labels.shape

In [ ]:
# importing and normalizing test data
test_data = np.genfromtxt('/Users/yerbol/Desktop/CS698/assignments/hw2/MNIST_X_test.csv', delimiter=',')
# row_sums = test_data.sum(axis=1)
# test_data = test_data / row_sums[:, np.newaxis]
print test_data.shape
pca = PCA(n_components=100)
test_data = pca.fit_transform(test_data)
print test_data.shape



In [ ]:
# importing and normalizing test labels
test_labels = np.genfromtxt('/Users/yerbol/Desktop/CS698/assignments/hw2/MNIST_y_test.csv', delimiter=',')

In [ ]:
print trn_data.shape
print trn_labels.shape
print test_data.shape
print test_labels.shape


In [ ]:
summ = 0
X_train_class = []
for c in range(10):
    y_c = np.where(trn_labels == c)
    trn_data_c = trn_data[y_c]
    X_train_class.append(trn_data_c)
    print X_train_class[c].shape

In [ ]:
def GMM(X, K):
    n = X.shape[0]
    d = X.shape[1]

    max_iter = 50 # maximum number of iterations
    
    ###### initialization of parameters:    
    
    # initializing means to random elements from X
    mu = X[np.random.choice(n, K), :].reshape((K,d)) # ~R^k*d
    S = np.random.rand(K, d) # ~R^k*d
    S = S + 1.1
    pi = np.random.rand(K)   
    pi = pi/np.sum(pi) # ~R^1*k 
    
    ###### Initializing log responsibility and responsibility matrices
    log_r = np.ndarray([n, K]) # ~R^n*k
    r = np.ndarray([n, K]) # ~R^n*k

    ###### list of log likelihoods
    l = []
    
    ###### Expectation Maximization        
    for iteration in range(max_iter):

        print "\n\n  iteration", iteration, ": "
        
        #### E-step (guessting hidden variables)
        # filling out log_r matrix for each Gaussian mode K (K styles of writing the digit)
        for k in range(K):    
            log_r[:,k] = np.log(pi[k]) - 0.5*np.sum(np.log(S[k,:])) - 0.5 * np.multiply((X - mu[k]), (X - mu[k])).dot(1.0/S[k,:])
              #R^n*1    R^n*1 (broadcasting)  R^n*1(broadcasting)                             R^n*d                 R^d*1
        
        
        # Shifting all logs by maximum among k's (maximum in erach row)
        log_r_max_i = log_r.max(axis = 1).reshape(n,1)
        log_r = log_r - log_r_max_i
        
        
        # matrix r with powers shifted by maximum power among k's
        r = np.exp(log_r)
        
        # normalize each row of r the sum of the row 
        r_i = np.sum(r, 1).reshape(n,1)
        r = np.divide(r, r_i) # elementwise division

        # negative log likelihood
        l_iter = -np.sum(np.log(r_i))
        l.append(l_iter)
        print('   log likelihood', l_iter)
        print("   difference", np.abs(l[iteration] - l[iteration-1]))
    
        if (iteration > 1 and np.abs(l[iteration] - l[iteration-1]) <= 10**-5 * np.abs(l[iteration])):
            break

        #### M-step (updating parameters of the model)
        r_k = np.sum(r, 0)
        pi = r_k / n
        mu = np.divide(r.T.dot(X), r_k.reshape(K,1))
        
        for k in range(K):
            for j in range(d):
                rx_sq = 0
                for i in range(n):
                    rx_sq += r[i,k] * X[i,j]**2
                S[k,j] = rx_sq/r_k[k] - (mu[k,j]**2) + 0.000001
                
    return mu, S, pi

In [ ]:
K = 5
GMM(X_train_class[0], K) # TEST

In [ ]:
# Probability of each class
num_samples = len(trn_labels)
prob_class = []
for c in range(10):
    y_c = np.where(trn_labels == c)
    prob_class.append(1.0*len(y_c[0]) / num_samples)
print prob_class

In [ ]:
# Extracting GMM's for each class
K = 5

pi_all = []
S_all = []
mu_all = []

for c in range(10):
    print "-"*20
    print " "*5, c
    print "-"*20
    mu_c, S_c, pi_c = GMM(X_train_class[c], K)
    mu_all.append(mu_c)
    S_all.append(S_c)
    pi_all.append(pi_c)

In [ ]:
# Testing
number_of_tests = 10000
correct = 0
K = 5
from scipy.stats import multivariate_normal
gaussian = np.zeros(10)
prediction_prob = np.zeros(10)

for i in range(number_of_tests):
    if i%100 == 0:
        print i
    # for each class
    for c in range(10):
        # for each gaussian multivariate in GMM
        for k in range(K):
            # find gaussian multivariate
            gaussian[k] = pi_all[c][k]*multivariate_normal.pdf(test_data[i], mean=mu_all[c][k], cov=np.diagflat(S_all[c][k]))
        prediction_prob[c] = np.sum(gaussian)*(prob_class[c])
    prediction = np.argmax(prediction_prob)
    if (prediction == test_labels[i]):
        correct+=1
print "Percent correct:", 1.0*correct / number_of_tests
    
    
    